# Import

In [3]:
!pip install PyPDF2
!pip install pikepdf
!pip install textract
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr \
flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install --user -U nltk
!pip install tika
!pip install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
python-dev is already the newest version (2.7.15~rc1-1).
antiword is already the newest version (0.37-11build1).
lame is already the newest version (3.100-2).
pstotext is already the newest version (1.9-6build1).
swig is already the newest version (3.0.12-1).
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
unrtf is already the newest version (0.21.9-clean-3).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.8).
libxslt1-dev is already the newest version (1.1.29-5ubuntu0.3).
poppler-utils is already the ne

In [4]:
import os
from pathlib import Path

import pandas as pd
import PyPDF2
import pikepdf
import textract
import json
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer

from tika import parser
import yake

In [5]:
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Functions

In [17]:
def main(type=1):
    '''if tupe = 1, works with file. Otherwise with folder'''
    if type == 1:
      file_path = input("Enter a file path: ")
      print(text_recognition(file_path=file_path))
    else:
      folder_path = input("Enter a folder path: ")
      names = os.listdir(folder_path)
      for name in names:
        file_path = folder_path + '/' + name
        print(text_recognition(file_path=file_path)) 

In [18]:
def text_recognition(file_path):
    '''For recognition pdf and convert to txt format. Also  convert doc/docx formats to txt'''
  
    text_file_name = f'result_{Path(file_path).stem}.txt' #name
    
    result = []
    new_text = ''

    if Path(file_path).suffix == '.pdf':
      try:
        pdf_file = open(file_path, 'rb')
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
      except:
        pdf_file = pikepdf.Pdf.open(file_path, allow_overwriting_input=True).save(file_path)
        pdf_file = open(file_path, 'rb')
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
      
      page = read_pdf.getPage(0)
      text = page.extract_text()

      text = text_lemma(text_preparation(text))

      with open(text_file_name, "w") as file:
        file.write(" ".join(text))
      
    elif Path(file_path).suffix == '.doc':
      parsed = parser.from_file(file_path)
      result.append(parsed.get('content'))
      text = text_lemma(text_preparation(result[0]))

      with open(text_file_name, "w") as file:
        file.write(" ".join(text))

    elif Path(file_path).suffix == '.docx':
      text = textract.process(file_path, extension='docx').decode('utf-8')
      text = text_lemma(text_preparation(text))

      with open(text_file_name, "w") as file:
        file.write(" ".join(text))

    else:
      pass
    

    try:
      df_creator(os.path.dirname(file_path)+'/'+text_file_name)
    except:
      pass
    
    '''try:
      os.remove(file_path)
    except:
      pass  '''

    return f"Result wrote into {text_file_name}"

In [19]:
def text_preparation(text):  
    text = re.sub(r'[^а-яА-Я ]', ' ', text)
    text = text.lower()
    return text

In [20]:
def keyword(file_path):
  with open(file_path, "r") as file:
    text = file.read()
  extractor = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 2,          # максимальное количество слов в фразе
    dedupLim = 0.9, # порог похожести слов
    top = 10        # количество ключевых слов
  )
  keywords = []
  for i in range(len(extractor.extract_keywords(text))):
    keywords.append(extractor.extract_keywords(text)[i][0])
  

  return " ".join(keywords)

In [21]:
def text_lemma(text):
  result =  filtr(lemmatize(tokeniser(text)))
  return result

In [22]:
def filtr(tokens):
    stop_words = stopwords.words("russian")
    stop_words.append(['настоящий договор', 'настоящего договора', 'настоящим договором', 'настоящему договору',\
                  'настоящего', 'договора', 'настоящим', 'договором ', 'договорy', 'к', 'на', '...'])
    filtered_tokens = []
    for token in tokens:
      if token not in stop_words:
          filtered_tokens.append(token)
    return filtered_tokens

In [23]:
def lemmatize(tokens):
    try:
        wnl = nltk.WordNetLemmatizer()
    except LookupError:
        nltk.download('wordnet')
        wnl = nltk.WordNetLemmatizer()
    return [wnl.lemmatize(t) for t in tokens] 

In [24]:
def tokeniser(text):
  tokens = word_tokenize(text, language="russian")
  return tokens

In [25]:
def df_creator(path):
  with open(path, "r") as file:
    file_text = file.read()
  df.loc[ len(df.index )] = [Path(path).name.split('_')[-1].split('.')[0], file_text, keyword(path)]
  
  #os.remove(path) #Удаляет ненужный файл

  return df

# Work process


In [15]:
df = pd.DataFrame(columns=['name', 'text', 'keywords'])

In [16]:
main(type=0)

Enter a folder path: /content
Result wrote into result_.config.txt
Result wrote into result_855c8450351c98a25ea2ac22a0256c7f.txt
Result wrote into result_35b6a0f57d909507c5aa9a8972b15f35.txt
Result wrote into result_6babd660f1a9192017a06b7e93bea919.txt
Result wrote into result_2b408919fd9833dc3f6892aad753f21f.txt
Result wrote into result_214d620d9c54bc83111277dd872d3cb2.txt
Result wrote into result_result_86d6d0062e91651ccc3d3ce35b162bf8.txt
Result wrote into result_result_a7966d0674c94a6e752e3a5832edb2a9.txt
Result wrote into result_2fd747f38e30ae7ce1c9d6e3b907ac5d.txt
Result wrote into result_.ipynb_checkpoints.txt
Result wrote into result_result_7ce2d613ac62ec5e4ef3d988c1f91582.txt
Result wrote into result_result_b7acf29be1d3e4b9cb529683e676fadb.txt
Result wrote into result_result_bdfb262c00991dddebfc5d4a87604284.txt
Result wrote into result_result_453b2cb6e4ba7b8d1a8382773880d313.txt
Result wrote into result_result_2351cf256df1b683a425211c1ed0e848.txt
Result wrote into result_19e2b

In [ ]:
path = '/content/classes.json'
with open(path) as file:
  dict_train = json.load(file)
cls_dict = {'Договоры для акселератора/Договоры поставки':'1',
            'Договоры для акселератора/Договоры оказания услуг':'2',
            'Договоры для акселератора/Договоры подряда': '3',
            'Договоры для акселератора/Договоры аренды':'4',
            'Договоры для акселератора/Договоры купли-продажи':'5'
    
}
train = pd.DataFrame.from_dict(dict_train,  orient='index').reset_index()
train[0] = train[0].map(lambda x: cls_dict[x])
train.rename(columns = {'index' : 'name'}, inplace=True)
train.name = train['name'].apply(lambda x: x.split('.')[0])

In [72]:
res = df.merge(train, on=["name"], how='left')
res = res[['name','text','keywords',0]]
res.rename(columns = {0 : 'type'}, inplace=True)
df_for_train = res
filepath = Path('df_for_train.csv')  
df_for_train.to_csv(filepath, index=False)